# Find ROIS

Many of the pixels in the frames from all camera views cant possibly contain a fish. Regions outside of the tank, for example.
The actual pixel region that can contain fish may vary experiment to experiment, as the interior cage may move, the cameras may have been moved, etc.

In this document you can interactively find the ROIs

In [1]:
import numpy as np
import h5py
import cv2
from joblib import load
import matplotlib.pyplot as plt
%matplotlib widget
import time
%load_ext autoreload
import os
%autoreload 2

from ipywidgets import Video, Image, VBox, Text
from sidecar import Sidecar
from IPython.display import display
from scipy.interpolate import interp1d
from scipy import interpolate
import numpy as np
import os
import cv2
import base64
import warnings
from IPython.core.debugger import set_trace


from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider, Layout, AppLayout
import ipywidgets as widgets


# Our custom-made imports
import sys
sys.path.append('./lib/')
from DataManagement import SplitdataManager, Filepaths, Calibration

# Set the experiment path

In [2]:
# --- set an experiment folder path, and make a splitdataManager ---- #
experiment_path = '/media/stephens-group/guest_drive/labelling/complete_shortfin_experiment/FishTank20200130_153857/'
splitman = SplitdataManager(experiment_path)

# Examine the frames

In [65]:
# --- get the first frame from the first movie for each camera ---- #

# set the frame index and SplitData index
sdIdx = 0 
fIdx = 0
movPaths = [splitman.splitdata_paths[sdIdx][i]+'.mp4' for i in range(3)]  # get paths for splitdata moves all cas
caps = [cv2.VideoCapture(video_path) for video_path in movPaths]          # open the movie caps
[cap.set(1,fIdx) for cap in caps]                                         # go to current frame
# grab the frame from each camera
if caps[0].isOpened() and caps[1].isOpened() and caps[2].isOpened():  
    # grab the 3 images
    ret_xz, frame_xz = caps[0].read()
    if not ret_xz:
        raise TypeError('xz movie for splitdata_idx={0} not opening'.format(splitdata_idx.value))
    ret_xy, frame_xy = caps[1].read()
    if not ret_xy:
        raise TypeError('xy movie for splitdata_idx={0} not opening'.format(splitdata_idx.value))
    ret_yz, frame_yz = caps[2].read()
    if not ret_yz:
        raise TypeError('yz movie for splitdata_idx={0} not opening'.format(splitdata_idx.value))
    frames = [frame_xz, frame_xy, frame_yz]
# close everything before we finish
[caps[capIdx].release() for capIdx in range(3)];


raw_frames = list(frames)

# --- Define ROIS ----- #
# tlc format: opencv (col, row) format with (0,0) in top left corner of image
# width: number of pixels along cols
# height: number of pixels along rows
                    
bbox_info = {"xz_tlc": [175, 175], "xz_width": 930, "xz_height": 820,
             "xy_tlc": [246, 120], "xy_width": 760, "xy_height": 755,
             "yz_tlc": [180, 180], "yz_width": 940, "yz_height": 800}



# --- Draw ROIS ----- #
crop_details = []
alpha = 0.3
for camIdx, camName in enumerate(['xz', 'xy', 'yz']):
    # get the frame
    frame = np.copy(frames[camIdx])
    # make a copy of the image and draw the rectangle
    overlay = np.copy(frames[camIdx])
    # get the tlc, width and height of ROI
    x, y = bbox_info[camName + '_tlc']
    w = bbox_info[camName + '_width']
    h = bbox_info[camName + '_height']
    crop_details.append([x, y, x+w, y+h])
    cv2.rectangle(overlay, (x, y), (x+w, y+h), (0, 200, 0), -1)  # A filled rectangle
    # combine the images in a weid
    edited_frame = cv2.addWeighted(overlay, alpha, frame, 1-alpha, 0)
    frames[camIdx] = edited_frame
                        
# ------------ plot the images -------------#

# XZ
camIdx = 0
fig1 = plt.figure()
ax1 = fig1.add_subplot(1, 1, 1)
ax1.set_title('XZ')
ax1.imshow(frames[camIdx])
fig2 = plt.figure()
ax2 = fig2.add_subplot(1, 1, 1)
ax2.set_title('XZ')
ax2.imshow( frames[camIdx][crop_details[camIdx][1]:crop_details[camIdx][3], crop_details[camIdx][0]:crop_details[camIdx][2]] )


camIdx = 1
fig3 = plt.figure()
ax3 = fig3.add_subplot(1, 1, 1)
ax3.set_title('XY')
ax3.imshow(frames[camIdx])
fig4 = plt.figure()
ax4 = fig4.add_subplot(1, 1, 1)
ax4.set_title('XY')
ax4.imshow( frames[camIdx][crop_details[camIdx][1]:crop_details[camIdx][3], crop_details[camIdx][0]:crop_details[camIdx][2]] )


camIdx = 2
fig5 = plt.figure()
ax5 = fig5.add_subplot(1, 1, 1)
ax5.set_title('YZ')
ax5.imshow(frames[camIdx])
fig6 = plt.figure()
ax6 = fig6.add_subplot(1, 1, 1)
ax6.set_title('YZ')
ax6.imshow( frames[camIdx][crop_details[camIdx][1]:crop_details[camIdx][3], crop_details[camIdx][0]:crop_details[camIdx][2]] )





Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [69]:
bbox_info

{'xz_tlc': [175, 175],
 'xz_width': 930,
 'xz_height': 820,
 'xy_tlc': [246, 120],
 'xy_width': 760,
 'xy_height': 755,
 'yz_tlc': [180, 180],
 'yz_width': 940,
 'yz_height': 800}